In [ ]:
from lambdamart import LambdaMART 
import numpy as np
import pickle
import pandas as pd

from tqdm import tqdm_notebook
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split

# Calculates via LambdaMART the MRR (& NDCG)

In [2]:
with open('feature_df.pickle', 'rb') as f:
    feature_vec = pickle.load(f)

In [3]:
splitpoint = int(0.8*len(feature_vec))

train_df = feature_vec.iloc[:splitpoint, :]
test_df = feature_vec.iloc[splitpoint:, :]

In [4]:
NUMBER_OF_TREES = 300

model = LambdaMART(
    training_data = train_df.to_numpy(), 
    number_of_trees = NUMBER_OF_TREES, 
    learning_rate = 0.01)
model.fit()

model.save('lambdamart_300trees')

Tree0


In [5]:
# model = LambdaMART()
# model.load('lambdamart_300trees.lmart')

In [6]:
test_data = test_df.to_numpy()
average_ndcg, predicted_scores = model.validate(test_data, k=10)
predicted_scores = model.predict(test_data[:,1:])

In [ ]:
test_by_qid = {}

counter = 1
for row in test_df.itertuples():
    qid = row.qid
    clear_output(wait=True)
    display('Iteration '+str(counter)+' of '+ str(len(test_df)))
    counter += 1
    
    if qid in test_by_qid:
        test_by_qid[qid].append(np.array(row))
    else:
        test_by_qid[qid] = [np.array(row)]

In [ ]:
def sort_list(list1, list2): 
  
    zipped_pairs = zip(list1, list2) 
  
    z = [x for _, x in sorted(zipped_pairs, key=lambda b:b[0])] 
      
    return z 

mrrs = []
for key in test_by_qid:
    candidates = np.array(test_by_qid[key])

    predictions = model.predict(candidates[:,2:])
    sortedList = sort_list(predictions, candidates)

    for i in range(len(sortedList)):
        label = sortedList[i][1]
        mrr = 0
        if label == 1:
            mrr = 1 / (i+1)
            break
    mrrs.append(mrr)    

In [ ]:
MRR = np.mean(mrrs)